<a href="https://colab.research.google.com/github/vineetyadav/kaggle_experiment/blob/main/Clinical_trails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Clinical Patient Notes Kagggle 


# New Section

# Loading DataSet in google colab 

Install Kaggle Library 

In [1]:
! pip install kaggle


 Create a directory named “.kaggle”

In [2]:
! mkdir ~/.kaggle

Copy Kaggle Json in directory 

In [3]:
! cp kaggle.json ~/.kaggle/


Provide permission to File 

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

Downloading Competitions dataset:

In [5]:
! kaggle competitions download -c nbme-score-clinical-patient-notes

 51% 5.00M/9.83M [00:00<00:00, 51.5MB/s]
100% 9.83M/9.83M [00:00<00:00, 88.7MB/s]


In [ ]:
!ls


features.csv  patient_notes.csv.zip  sample_submission.csv  train.csv
kaggle.json   sample_data	     test.csv


In [6]:
!unzip /content/nbme-score-clinical-patient-notes.zip 

Archive:  /content/nbme-score-clinical-patient-notes.zip
  inflating: features.csv            
  inflating: patient_notes.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Exploring Dataset

## import all required packages 

In [7]:
import pandas as pd
import matplotlib.pyplot as mlpt 
import numpy as np
import os 


Exploring training, test and original dataset. 

In [8]:
dataset = pd.read_csv("patient_notes.csv")
dataset.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [9]:
train_dataset = pd.read_csv("train.csv")
train_dataset.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [10]:
test_dataset = pd.read_csv("test.csv")
test_dataset.head()

,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


Merge train dataset with orignal dataset 

In [11]:
merged_train_dataset = train_dataset.merge(dataset,on="pn_num",how="inner")
merged_train_dataset.head()

,id,case_num_x,pn_num,feature_num,annotation,location,case_num_y,pn_history
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],0,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],0,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,['chest pressure'],['203 217'],0,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",0,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],0,HPI: 17yo M presents with palpitations. Patien...


In [12]:
loc = list(merged_train_dataset.loc[(merged_train_dataset.pn_num==224) & (merged_train_dataset.location!='[]'), 'location'].str.replace("['", "", regex=False).str.replace("']", "", regex=False))


In [13]:
import spacy
from spacy import displacy
nlp = spacy.blank('en')

In [14]:
text = merged_train_dataset[merged_train_dataset.pn_num==224].pn_history.values[0]
doc = nlp.make_doc(text)
ents = []
for l in loc:
    start, end = l.split(' ')
    ent = doc.char_span(int(start), int(end), label='annotation')
    ents.append(ent)
doc.ents = ents
color = {"Annotation": '#A32EFF'}
displacy.render(doc, style="ent", jupyter=True, options={'colors': color})

##Merge Test dataset with orignal dataset 

In [15]:
merged_test_dataset = test_dataset.merge(dataset,on="pn_num",how="inner")
merged_test_dataset.head()

,id,case_num_x,pn_num,feature_num,case_num_y,pn_history
0,00016_000,0,16,0,0,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,0,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,0,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,0,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,0,HPI: 17yo M presents with palpitations. Patien...


In [16]:
merged_train_dataset['location'] = merged_train_dataset['location'].apply(ast.literal_eval)
merged_train_dataset.head()


NameError: ignored

In [17]:
from ast import literal_eval
merged_train_dataset["location_list"] = [literal_eval(x) for x in merged_train_dataset["location"]]

In [18]:
merged_train_dataset["location_new"] = merged_train_dataset["location"].replace("[","",regex=False).replace("]","",regex=False)
merged_train_dataset["location_new"]

0                 ['696 724']
1                 ['668 693']
2                 ['203 217']
3        ['70 91', '176 183']
4                 ['222 258']
                 ...         
14295                      []
14296                      []
14297             ['274 282']
14298             ['421 437']
14299             ['314 330']
Name: location_new, Length: 14300, dtype: object

In [19]:
merged_train_dataset["location_list"]

0               [696 724]
1               [668 693]
2               [203 217]
3        [70 91, 176 183]
4               [222 258]
               ...       
14295                  []
14296                  []
14297           [274 282]
14298           [421 437]
14299           [314 330]
Name: location_list, Length: 14300, dtype: object

In [20]:
! pip install spacy

In [21]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [22]:
entity ="Phrase"
import ast
#print(merged_train_dataset.columns)
#merged_train_dataset.apply(lambda x: (, 'cols': x.tolist()[1:]), axis=1).tolist()
x = merged_train_dataset.apply(lambda x: (x['pn_history'],{'entries':[(ast.literal_eval(x['location']),entity)]}),axis=1)
print(x[0][1])



{'entries': [(['696 724'], 'Phrase')]}


In [34]:
def train_on_one_paragraph(pn_num):
  paragraph_subset = merged_train_dataset[merged_train_dataset["pn_num"]==pn_num]
  locations = paragraph_subset["location"].apply(eval)
  text = paragraph_subset["pn_history"][0]
  #print("Hi",text)
  ents = []
  for location in locations:
        for i in range(len(location)):
            for loc in location:
                val = loc.split()
                val = [int(each) for each in val]
                ents.append((val,"Entity"))
  final_entity_dict = {"entries":ents}
  final_res = (text,final_entity_dict)
  return final_res
train_on_one_paragraph(16)

('HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms', {'entries': [([696, 724], 'Entity'), ([668, 6

In [ ]:

def transform_column(df):
  


SyntaxError: ignored